# Export a CSV with volume data (ESV, EDV, EF, split) from algorithms and VolumeTracings

In [7]:
# Dictionary with volumes from algorithms and VolumeTracings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import loader
from algorithms import funcs as funcs
import config
from algorithms import volume_tracings_calculations as tracings
from ast import literal_eval
from tqdm import tqdm
import operator

tracings_path = "/Users/ishan/Documents/Stanford/EchoData/VolumeTracings.csv"
frames_path = "/Users/ishan/Documents/Stanford/EchoData/Netherlands"

df = pd.read_csv(tracings_path)
df = df.astype(str).groupby(['FileName', 'Frame']).agg(','.join).reset_index() # group VolumeTracings.csv by FileName and Frame timing

def sortVolumesFromAlgo():
    all_volumes={}
    exception_files = 0
    
    PATH_TO_RAW_FRAMES_PARENT_DIR = os.path.join(frames_path) # frames path
    
    frame_df = pd.read_csv("/Users/ishan/Documents/Stanford/EchoData/Frame Timing Predictions.csv")
    for i in tqdm(range(len(frame_df))): # iterates through each row of data frame
        try:
            videoName = frame_df.iloc[i, 1] # name of video
            ESV_frameNumber = frame_df.iloc[i, 2] # ESV timing for clip
            EDV_frameNumber = frame_df.iloc[i, 3] # EDV timing for clip

            ESV_OUTPUT_FRAME_NAME = videoName.split(".avi")[0] + "/" + str(ESV_frameNumber) + ".png" # concatenate video name with frame number as file name
            EDV_OUTPUT_FRAME_NAME = videoName.split(".avi")[0] + "/" + str(EDV_frameNumber) + ".png" # concatenate video name with frame number as file name

            ESV_FRAMES_PATH = os.path.join(PATH_TO_RAW_FRAMES_PARENT_DIR, ESV_OUTPUT_FRAME_NAME) # path to each video
            EDV_FRAMES_PATH = os.path.join(PATH_TO_RAW_FRAMES_PARENT_DIR, EDV_OUTPUT_FRAME_NAME) # path to each video

            if os.path.exists(ESV_FRAMES_PATH) and os.path.exists(EDV_FRAMES_PATH):
                ES_volumes, *_ = funcs.calculateVolumeMainAxisTopShift(ESV_FRAMES_PATH, 20, 1, "Method of Disks")
                ED_volumes, *_ = funcs.calculateVolumeMainAxisTopShift(EDV_FRAMES_PATH, 20, 1, "Method of Disks")

                all_volumes[videoName] = [ES_volumes[0], ED_volumes[0]]
        except:
            exception_files += 1
        
    return all_volumes

def calculateVolumesForEachFrame(inputFolderName="Netherlands"):
    """Function to extract frames from input video file
    and save them as separate frames in an output directory.
    Args:
        inputFolderName: Input directory to where frames are located.
    Returns:
        None
    """
    
    root = "/Users/ishan/Documents/Stanford/EchoData/"
    
    volumeDict = {}
    failed_frames = 0
    
    framesPath = os.path.join(root, inputFolderName)
        
    # Calculate Volumes
    for video in tqdm(os.listdir(framesPath)):
        if "0" in video:
            if video not in volumeDict:
                volumeDict[video] = {}

            for frame in os.listdir(os.path.join(framesPath, video)):
                if ".png" in frame:
                    try:
                        framePath = os.path.join(framesPath, video, frame)
                        volumes, *_ = funcs.calculateVolumeMainAxisTopShift(framePath, 20, pointShifts=1, method="Method of Disks") # 0th shift for regular volume
                        volumeDict[video][os.path.splitext(frame)[0]] = volumes[0]
                    except:
                        failed_frames += 1
    return volumeDict

def sortFrameVolumeTracings():
    calculatedVolumeFromGroundTruth={}
    
    for i in tqdm(range(len(df))):
        videoName = df.iloc[i, 0] + ".avi"
        x1 = list(literal_eval(df.iloc[i, 2])) # x1 coords
        y1 = list(literal_eval(df.iloc[i, 3])) # y1 coords
        x2 = list(literal_eval(df.iloc[i, 4])) # x2 coords
        y2 = list(literal_eval(df.iloc[i, 5])) # y2 coords

        number = len(x1) - 1

        maxX1, maxY1, maxX2, maxY2, lowerInterceptAveragePoints, higherInterceptAveragePoints = tracings.calcParallelAndMaxPoints(x1, y1, x2, y2)

        if number < 22:
            ground_truth_volume = funcs.volumeMethodOfDisks(maxX1, maxY1, maxX2, maxY2, number, lowerInterceptAveragePoints, higherInterceptAveragePoints)
            
            if videoName not in calculatedVolumeFromGroundTruth:
                calculatedVolumeFromGroundTruth[videoName] = []
                
            calculatedVolumeFromGroundTruth[videoName].append(ground_truth_volume)
    return calculatedVolumeFromGroundTruth

#algorithm_values = sortVolumesFromAlgo()
all_volumes = calculateVolumesForEachFrame()
#volume_tracings = sortFrameVolumeTracings()


100%|██████████| 750/750 [03:12<00:00,  3.89it/s]


In [16]:
# Creates CSV with All Volumes

volumeDict, dataList = {}, []
def get_key(key):
    try:
        return int(key)
    except ValueError:
        return key
    
for study in all_volumes:
    a = all_volumes[study]
    sorted_values = sorted(a.items(), key=lambda t: get_key(t[0]))
    for volumeData in sorted_values:
        frameNumber = volumeData[0]
        volume = volumeData[1]
        volumeDict = {"Video Name": study, "Frame": frameNumber, "Volume": volume}
        
        dataList.append(volumeDict)

df = pd.DataFrame(dataList) # convert to dataframe
df.to_csv("/Users/ishan/Documents/Stanford/EchoData/All Netherlands Volumes.csv") # export to CSV
        

In [2]:
# Returns dictionary with split

filelist_path = "/Users/ishan/Documents/Stanford/EchoData/FileList.csv"
filelist_df = pd.read_csv(filelist_path)

def sortVolumesFromFileList(root=config.CONFIG.DATA_DIR):
    givenTrueDict={}
        
    for i in tqdm(range(len(filelist_df))):
        videoName = filelist_df.iloc[i, 0]
        split = filelist_df.iloc[i, 8]
        
        if videoName not in givenTrueDict:
            givenTrueDict[videoName] = []
        givenTrueDict[videoName] = split
        
    return givenTrueDict

split = sortVolumesFromFileList()

100%|██████████| 10030/10030 [00:00<00:00, 19703.45it/s]


In [12]:
# Exports CSV with volume data from human tracings and algorithms

dataList = []

for videoName in algorithm_values:
    #if videoName in split:
    #current_split = split[videoName] # split of the video (train, val, or test)

    #volumeData = volume_tracings[videoName]
#     ground_truth_ESV = min(volumeData)
#     ground_truth_EDV = max(volumeData)
#     ground_truth_EF = (1 - (ground_truth_ESV/ground_truth_EDV)) * 100
    #if videoName in algorithm_values:
    volumes = algorithm_values[videoName]

    if len(volumes) > 1:
        EDV = max(volumes)
        ESV = min(volumes)
        EF = (1 - (ESV/EDV)) * 100

        if EF != 0 and ground_truth_EF != 0:
#                     calculatedData = {"Video Name": videoName, "Split": current_split, 
#                                       "Human Tracings EF": ground_truth_EF,"Human Tracings ESV": ground_truth_ESV,
#                                       "Human Tracings EDV": ground_truth_EDV,"Algorithm EF": EF, "Algorithm ESV": ESV,
#                                       "Algorithm EDV": EDV}
            calculatedData = {"Video Name": videoName, "EF": EF, "ESV": ESV, "EDV": EDV}
            dataList.append(calculatedData)

export_df = pd.DataFrame(dataList) # convert to dataframe
export_path = os.path.join("/Users/ishan/Documents/Stanford/EchoData/Netherlands Volume Data.csv") # path to export

export_df.to_csv(export_path) # export to CSV

NameError: name 'split' is not defined